In [187]:
import torch
import numpy as np
import pandas as pd
import os

rdir = '/home/hyeryungson'

if os.getcwd() != f'{rdir}/mucoco':
    print(f"change dir to '{rdir}/mucoco'")
    os.chdir(f'{rdir}/mucoco')

from notebooks.utils.load_ckpt import define_model

In [188]:
test_all = pd.read_json('data/toxicity/jigsaw-unintended-bias-in-toxicity-classification/fine-grained/test.jsonl',
                    lines=True)

In [189]:
print("num test samples", test_all.shape[0])

num test samples 3998


In [190]:
print(f"checking duplicates... {test_all.duplicated(subset=['text']).sum()} duplicated texts")

checking duplicates... 0 duplicated texts


In [191]:
test_all['seq_len'] = test_all['text'].apply(lambda x: len(x.split()))

In [192]:
test_all['seq_len_q'] = pd.qcut(test_all['seq_len'], 4)

In [193]:
test_all['seq_len_q'].value_counts()

(0.999, 16.0]    1005
(16.0, 33.0]     1001
(33.0, 66.0]     1000
(66.0, 196.0]     992
Name: seq_len_q, dtype: int64

In [42]:
test = test_all.loc[test_all['toxicity']==1.0].copy()

In [48]:
samples_1 = test.groupby('seq_len_q').sample(5, random_state=999)

In [76]:
samples_1.seq_len

2665     13
2420     12
3937     14
3991     16
3279      7
3880     27
2877     30
3988     21
3183     27
2339     21
2561     47
3173     50
3126     40
2863     50
2591     56
3627    155
3825    177
2922    108
3228    180
3629    122
Name: seq_len, dtype: int64

In [80]:
samples_1.loc[2591, 'text']

'There are too many low white racists in Canada. This is a brave soldier who gave a somewhat exaggerated account of his role in one battle. Big deal. Ordinarily, it would be shrugged off with a wink and a tolerant grin. But the man is brown and so all the white skunks have leapt on him.'

In [51]:
test_9x = test_all.loc[(test_all['toxicity']>=0.9) & (test_all['toxicity']<1.0)].copy()

In [57]:
samples_9x = test_9x.groupby('seq_len_q').sample(5, replace=True, random_state = 999)

In [59]:
samples_9x = samples_9x.drop_duplicates()

In [62]:
print('len of samples_9x:', len(samples_9x))

len of samples_9x: 16


In [63]:
test_9x = test_9x.drop(samples_9x.index)

In [65]:
samples_9x_ = test_9x.sample(20-len(samples_9x), random_state=999)

In [66]:
samples_9x = pd.concat([samples_9x, samples_9x_], axis=0)

In [68]:
samples_9x.seq_len

2134      7
3925      6
2535      9
2284      8
2728      4
2211     26
3756     32
2507     17
2115     20
3085     21
3048     38
3067     58
2337     80
2645    145
3658     80
2345    139
3392     16
2907     11
3483     32
2785      9
Name: seq_len, dtype: int64

In [75]:
samples_9x.loc[2345, 'text']

'the ignorance of this issue from lunatic lefties reminds me of stupid people with no clue of how business or an economy works so they being empty of facts turn to rediculous attempt comparisons that have no connection to the subject to spew some nonsense, see lunatic, lets say GM was forced to increase labour costs 32% overnight, you know that chev cruz will then cost 100K, but now you cant afford it cause you didnt get a raise as you work above the min wage like so many do already, so gm cant sell them so they have to cut costs to reduce the price and that means PEOPLE GET LAID OFF, but you are all ok with that, ignoring that this forced 32% increase in the min wage means poor people will now have no income soon'

In [67]:
samples_9x['seq_len']

2134      7
3925      6
2535      9
2284      8
2728      4
2211     26
3756     32
2507     17
2115     20
3085     21
3048     38
3067     58
2337     80
2645    145
3658     80
2345    139
3392     16
2907     11
3483     32
2785      9
Name: seq_len, dtype: int64

In [83]:
fin_samples = pd.concat([samples_1, samples_9x],axis=0)

In [85]:
del fin_samples['seq_len_q']

In [132]:
fin_samples = fin_samples.reset_index(drop=True)

In [133]:
# 교수님과 얘기했던 UI = 일단은 word 단위로 라벨을 받아보자.
fin_samples['text_words'] = fin_samples.text.str.split()

In [134]:
fin_samples

,text,toxicity,seq_len,text_words
0,"How droll, troll !\n\nThought that Kochbertan ...",1.000000,13,"[How, droll,, troll, !, Thought, that, Kochber..."
1,Murder is the ultimate crime. If he is guilty...,1.000000,12,"[Murder, is, the, ultimate, crime., If, he, is..."
2,The man has serious mental health issues. The...,1.000000,14,"[The, man, has, serious, mental, health, issue..."
3,Neither are you - troll behavior is not dialog...,1.000000,16,"[Neither, are, you, -, troll, behavior, is, no..."
4,Time to kick them to the curb.,1.000000,7,"[Time, to, kick, them, to, the, curb.]"
5,"Yeah, but they got a lot more attention this w...",1.000000,27,"[Yeah,, but, they, got, a, lot, more, attentio..."
6,Nothing says they'll actually launch something...,1.000000,30,"[Nothing, says, they'll, actually, launch, som..."
7,"Mr. Trump is a classic bully, and bullies thri...",1.000000,21,"[Mr., Trump, is, a, classic, bully,, and, bull..."
8,"Hopefully I will go someday, but that is when ...",1.000000,27,"[Hopefully, I, will, go, someday,, but, that, ..."
9,why don't we ban dummies in the government - t...,1.000000,21,"[why, don't, we, ban, dummies, in, the, govern..."


In [135]:
words = pd.DataFrame(fin_samples['text_words'].tolist())

In [136]:
lens = pd.DataFrame(fin_samples['text_words'].apply(len)).rename(columns={'text_words': 'num_words'})

In [157]:
df_for_labelling = pd.concat([fin_samples[['text', 'toxicity']], lens, words], axis=1)

In [158]:
df_for_labelling

,text,toxicity,num_words,0,1,2,3,4,5,6,...,170,171,172,173,174,175,176,177,178,179
0,"How droll, troll !\n\nThought that Kochbertan ...",1.000000,13,How,"droll,",troll,!,Thought,that,Kochbertan,...,None,None,None,None,None,None,None,None,None,None
1,Murder is the ultimate crime. If he is guilty...,1.000000,12,Murder,is,the,ultimate,crime.,If,he,...,None,None,None,None,None,None,None,None,None,None
2,The man has serious mental health issues. The...,1.000000,14,The,man,has,serious,mental,health,issues.,...,None,None,None,None,None,None,None,None,None,None
3,Neither are you - troll behavior is not dialog...,1.000000,16,Neither,are,you,-,troll,behavior,is,...,None,None,None,None,None,None,None,None,None,None
4,Time to kick them to the curb.,1.000000,7,Time,to,kick,them,to,the,curb.,...,None,None,None,None,None,None,None,None,None,None
5,"Yeah, but they got a lot more attention this w...",1.000000,27,"Yeah,",but,they,got,a,lot,more,...,None,None,None,None,None,None,None,None,None,None
6,Nothing says they'll actually launch something...,1.000000,30,Nothing,says,they'll,actually,launch,something.,A,...,None,None,None,None,None,None,None,None,None,None
7,"Mr. Trump is a classic bully, and bullies thri...",1.000000,21,Mr.,Trump,is,a,classic,"bully,",and,...,None,None,None,None,None,None,None,None,None,None
8,"Hopefully I will go someday, but that is when ...",1.000000,27,Hopefully,I,will,go,"someday,",but,that,...,None,None,None,None,None,None,None,None,None,None
9,why don't we ban dummies in the government - t...,1.000000,21,why,don't,we,ban,dummies,in,the,...,None,None,None,None,None,None,None,None,None,None


In [159]:
df_for_labelling.to_excel('notebooks/locate/labelling/jigsaw_geq_0.9.xlsx')

In [174]:
df_for_labelling = pd.read_excel('notebooks/locate/labelling/jigsaw_geq_0.9.xlsx', index_col=0)

In [175]:
df_for_labelling

,text,toxicity,num_words,0,1,2,3,4,5,6,...,170,171,172,173,174,175,176,177,178,179
0,"How droll, troll !\n\nThought that Kochbertan ...",1.000000,13,How,"droll,",troll,!,Thought,that,Kochbertan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Murder is the ultimate crime. If he is guilty...,1.000000,12,Murder,is,the,ultimate,crime.,If,he,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The man has serious mental health issues. The...,1.000000,14,The,man,has,serious,mental,health,issues.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Neither are you - troll behavior is not dialog...,1.000000,16,Neither,are,you,-,troll,behavior,is,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Time to kick them to the curb.,1.000000,7,Time,to,kick,them,to,the,curb.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Yeah, but they got a lot more attention this w...",1.000000,27,"Yeah,",but,they,got,a,lot,more,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Nothing says they'll actually launch something...,1.000000,30,Nothing,says,they'll,actually,launch,something.,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Mr. Trump is a classic bully, and bullies thri...",1.000000,21,Mr.,Trump,is,a,classic,"bully,",and,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Hopefully I will go someday, but that is when ...",1.000000,27,Hopefully,I,will,go,"someday,",but,that,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,why don't we ban dummies in the government - t...,1.000000,21,why,don't,we,ban,dummies,in,the,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
df_for_labelling["type"] = 0

In [177]:
df_for_labelling["text_index"] = df_for_labelling.index

In [178]:
blanks_for_labelling = df_for_labelling.copy()
blanks_for_labelling["type"] = 1
blanks_for_labelling["text"] = np.nan
blanks_for_labelling.loc[:, 0:179] = None

In [179]:
df_for_labelling = pd.concat([df_for_labelling, blanks_for_labelling], axis=0)

In [180]:
df_for_labelling = df_for_labelling.sort_values(by=['text_index', 'type'], ascending=[True, True])

In [181]:
df_for_labelling = df_for_labelling.reset_index(drop=True)

In [182]:
df_for_labelling['type'].replace({0: 'text', 1: 'label'}, inplace=True)

In [183]:
df_for_labelling

,text,toxicity,num_words,0,1,2,3,4,5,6,...,172,173,174,175,176,177,178,179,type,text_index
0,"How droll, troll !\n\nThought that Kochbertan ...",1.000000,13,How,"droll,",troll,!,Thought,that,Kochbertan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,text,0
1,NaN,1.000000,13,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,label,0
2,Murder is the ultimate crime. If he is guilty...,1.000000,12,Murder,is,the,ultimate,crime.,If,he,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,text,1
3,NaN,1.000000,12,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,label,1
4,The man has serious mental health issues. The...,1.000000,14,The,man,has,serious,mental,health,issues.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,text,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,NaN,0.958501,11,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,label,37
76,Rubbish.\nMany of us fought long and hard to h...,0.909091,32,Rubbish.,Many,of,us,fought,long,and,...,None,None,None,None,None,None,None,None,text,38
77,NaN,0.909091,32,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,label,38
78,"You're a witless troll.\nSee, I'm right once ...",0.916667,9,You're,a,witless,troll.,"See,",I'm,right,...,None,None,None,None,None,None,None,None,text,39


In [184]:
from googletrans import Translator
from tqdm import tqdm
translator = Translator()
text_index = df_for_labelling['text_index'].unique()

for ix in tqdm(text_index):

    
    if df_for_labelling.loc[(df_for_labelling['text_index']==ix) & (df_for_labelling['type']=='label'), 'text'].isna().values[0]:
        try:
            _text = df_for_labelling.loc[(df_for_labelling['text_index']==ix) & (df_for_labelling['type']=='text'), 'text'].values[0]
            _trns_text = translator.translate(_text, dest="ko").text

            df_for_labelling.loc[(df_for_labelling['text_index']==ix) & (df_for_labelling['type']=='label'),
                          'text'] = _trns_text
        except:
            print(ix)
            print(_text)

100%|██████████| 40/40 [00:49<00:00,  1.23s/it]


In [185]:
df_for_labelling

,text,toxicity,num_words,0,1,2,3,4,5,6,...,172,173,174,175,176,177,178,179,type,text_index
0,"How droll, troll !\n\nThought that Kochbertan ...",1.000000,13,How,"droll,",troll,!,Thought,that,Kochbertan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,text,0
1,"어떻게 드러나, 트롤!\n\nKochbertan Neocons는 세금 납부를 믿지 ...",1.000000,13,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,label,0
2,Murder is the ultimate crime. If he is guilty...,1.000000,12,Murder,is,the,ultimate,crime.,If,he,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,text,1
3,"살인은 궁극적 인 범죄입니다.그가 유죄라면, 그는 교수형을 일으켜야합니다.",1.000000,12,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,label,1
4,The man has serious mental health issues. The...,1.000000,14,The,man,has,serious,mental,health,issues.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,text,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,중국인은 무엇을 할 것입니까? ..... Walmart에 쓰레기를 팔지 않습니까?,0.958501,11,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,label,37
76,Rubbish.\nMany of us fought long and hard to h...,0.909091,32,Rubbish.,Many,of,us,fought,long,and,...,None,None,None,None,None,None,None,None,text,38
77,쓰레기.\n우리 중 많은 사람들이 우리 학교에서 기독교를 버리기 위해 오랫동안 열심...,0.909091,32,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,label,38
78,"You're a witless troll.\nSee, I'm right once ...",0.916667,9,You're,a,witless,troll.,"See,",I'm,right,...,None,None,None,None,None,None,None,None,text,39


In [186]:
df_for_labelling.to_excel('notebooks/locate/labelling/jigsaw_geq_0.9.xlsx')

### 7.15.23 version

In [2]:
test_all = pd.read_json('data/toxicity/jigsaw-unintended-bias-in-toxicity-classification/fine-grained/test.jsonl',
                    lines=True)

In [3]:
print("num test samples", test_all.shape[0])

num test samples 3998


In [4]:
print(f"checking duplicates... {test_all.duplicated(subset=['text']).sum()} duplicated texts")

checking duplicates... 0 duplicated texts


In [15]:
test = test_all.loc[test_all['toxicity']>=0.5].copy()

In [16]:
print("num toxic test samples", test.shape[0])

num toxic test samples 55


In [10]:
# check distribution by toxicity
test = test.reset_index(drop=True)
test['toxicity_bin'] = pd.cut(test['toxicity'], np.arange(0.5, 1.1, 0.1), right=False)#, labels=["[0.5, 0.6)"])
test_=test.copy()
test.groupby('toxicity_bin').size()

toxicity_bin
[0.5, 0.6)    659
[0.6, 0.7)    498
[0.7, 0.8)    397
[0.8, 0.9)    300
[0.9, 1.0)     51
[1.0, 1.1)     55
dtype: int64

In [11]:
# 어떻게 하는게 가장 좋을까? 흠.. 
# 고려해야 할점.. 
# - 어떤 UI 가 라벨링하기 좋을까? -> 일단 그것부터 생각?
# - toxicity 값은 보여줄까? -> 일단 샘플을 다 뽑아놓고, 필터링해서 제공하면 되지 않을까? 일단 컬럼에 포함.

In [12]:
# 교수님과 얘기했던 UI = 일단은 word 단위로 라벨을 받아보자.
test['text_words'] = test.text.str.split()

In [13]:
test

,text,toxicity,toxicity_bin,text_words
0,You aren't that bright. Citizens were picking ...,0.500000,"[0.5, 0.6)","[You, aren't, that, bright., Citizens, were, p..."
1,References are constantly being made that ind...,0.800000,"[0.8, 0.9)","[References, are, constantly, being, made, tha..."
2,"""...these are the people.."" Those REAL people...",0.833333,"[0.8, 0.9)","[""...these, are, the, people.."", Those, REAL, ..."
3,This article is especially hilarious in light ...,0.600000,"[0.6, 0.7)","[This, article, is, especially, hilarious, in,..."
4,Absolutely not! We should never learn to live...,0.500000,"[0.5, 0.6)","[Absolutely, not!, We, should, never, learn, t..."
...,...,...,...,...
1955,I don’t see it happening because of all the pr...,0.750000,"[0.7, 0.8)","[I, don’t, see, it, happening, because, of, al..."
1956,Every time a lib opens his mouth more garbage ...,0.600000,"[0.6, 0.7)","[Every, time, a, lib, opens, his, mouth, more,..."
1957,the dealers don't have a hard time making mone...,0.738462,"[0.7, 0.8)","[the, dealers, don't, have, a, hard, time, mak..."
1958,My opinion is Trump keeps opening his mouth an...,0.600000,"[0.6, 0.7)","[My, opinion, is, Trump, keeps, opening, his, ..."


In [14]:
words = pd.DataFrame(test['text_words'].tolist())

In [15]:
lens = pd.DataFrame(test['text_words'].apply(len)).rename(columns={'text_words': 'num_words'})

In [16]:
df_for_labelling = pd.concat([test[['text', 'toxicity']], lens, words], axis=1)

In [18]:
df_for_labelling.to_excel('notebooks/locate/labelling/test_geq_0.5.xlsx')

In [81]:
df_for_labelling = pd.read_excel('notebooks/locate/labelling/test_geq_0.5.xlsx', index_col=0)

In [82]:
df_for_labelling["type"] = 0

In [83]:
df_for_labelling["text_index"] = df_for_labelling.index

In [84]:
blanks_for_labelling = df_for_labelling.copy()
blanks_for_labelling["type"] = 1
blanks_for_labelling.loc[:, 0:195] = None

In [85]:
df_for_labelling = pd.concat([df_for_labelling, blanks_for_labelling], axis=0)

In [86]:
df_for_labelling = df_for_labelling.sort_values(by=['text_index', 'type'], ascending=[True, True])

In [87]:
df_for_labelling = df_for_labelling.reset_index(drop=True)

In [88]:
df_for_labelling['type'].replace({0: 'text', 1: 'label'}, inplace=True)

In [90]:
df_for_labelling.columns

Index([      'text',   'toxicity',  'num_words',            0,            1,
                  2,            3,            4,            5,            6,
       ...
                188,          189,          190,          191,          192,
                193,          194,          195,       'type', 'text_index'],
      dtype='object', length=201)

In [91]:
df_for_labelling.to_excel('notebooks/locate/labelling/test_geq_0.5_v0.2.xlsx')

### Old code (06.08)

In [ ]:
test = pd.read_json('data/toxicity/jigsaw-unintended-bias-in-toxicity-classification/fine-grained/test.jsonl',
                    lines=True)

In [ ]:
test = test.loc[test['toxicity']>=0.5].copy()

In [3]:
## select toxic 0.5~1.0
test = test.reset_index(drop=True)
test['toxicity_bin'] = pd.cut(test['toxicity'], np.arange(0.5, 1.1, 0.1), labels=np.arange(6))
test_=test.copy()

## randomly select from each bin
tot_count=150
bin_count=6
test = test.groupby('toxicity_bin').sample(int(tot_count/bin_count), random_state=11)

## add 4 examples from toxicity 1
test = pd.concat([test, test_.loc[((~test_.index.isin(test.index)) & (test_.toxicity>0.99))].sample(4, random_state=11)], axis=0)

test.to_json('./notebooks/results/labelling_samples_test_geq0.5_150.jsonl', orient='records', lines=True)

In [6]:
model,config,tokenizer=define_model('/home/hyeryungson/mucoco/models_bak_contd/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/pytorch_model.bin',
                                 output_attentions=True, output_hidden_states=True)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializin

DEVICE:  cuda


In [7]:
tokenized=tokenizer(test['text'].tolist(),padding=True, return_tensors="pt", truncation=True)

In [8]:
config.max_position_embeddings

514

In [9]:
# model input size와 같아야할거같은데!
tokenized_ids=tokenized['input_ids']

In [10]:
res = []
for i in range(tokenized_ids.shape[0]):
    tmp_sent = tokenized_ids[i]
    tmp_toks = [tokenizer.decode(x) for x in tmp_sent]
    res.append(tmp_toks)

In [11]:
if os.path.exists('./notebooks/results/toxic_labels.txt'):
    with open('./notebooks/results/toxic_labels.txt','r') as f:
        contents = f.readlines()
    last_index = len(contents)/2
    print(f'starting from index {last_index}')
else:
    fptr = open('./notebooks/results/toxic_labels.txt', 'w')
    fptr.close()
    last_index=0

starting from index 4.0


In [49]:
for i in range(last_index, tokenized_ids.shape[0]):
    try:
        print(f"{test['toxicity'].tolist()[i]} ||||||||||| {''.join(res[i])}")
        for j in range(len(res[i])):
            if res[i][j] != '<|endoftext|>':
                print(f"{j} ------ {res[i][j]}")
        toxic_tokens=input("input comma separated list of tokens that are toxic >>")
        if '-' in toxic_tokens:
            start, end = toxic_tokens.split('-')    
            toxic_tokens = ','.join([str(x) for x in range(int(start.strip()), int(end.strip())+1,1)])
        print(f"List you typed: {toxic_tokens}")
        # toxic_tokens=toxic_tokens.split(",")
        fptr = open('./notebooks/results/toxic_labels.txt', 'a')
        fptr.write(''.join(res[i]) + '\n')
        fptr.write(toxic_tokens + '\n')
        fptr.close()
    except KeyboardInterrupt:
        fptr = open('./notebooks/results/toxic_labels.txt', 'a')
        fptr.write(i + '\n')
        fptr.close()
        last_index = i    
    

0.525 ||||||||||| as long as you cannot admit who your enemies are you are losing fast. these guys want you dead yet they get modern missiles from your army. WTF!!!!<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|e